## Experiments

Let's start playing around with our data

In [40]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification,AutoModel,AutoConfig
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from imblearn.under_sampling import RandomUnderSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read in training data
This dataset contains 100 annotated terms of service contracts, each row represents a sentence, which carries on it a label. The label corresponds to a different type of potential unfairness, as defined by the authors of CLAUDETTE, the previous paper from which this dataset came from. 

In [3]:
df = pd.read_csv('../data/dataset.csv')
df.head()

,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


Before we were trying binary classification, which wasn't producing great results, let's see if we can get better results using the individual types of labels as classified by the text. The logic here is that because each type of potential unfairness likely has some semantic differences, conglomerating them all into one made it difficult to pick them all out. 

# Multi-Label Preprocessing

In [4]:
df['labels'] = df.apply(lambda row: (1 if row['A'] == 1 else 2 if row['CH'] == 1 else 3 if row['CR'] == 1 else 4 if row['J'] == 1 else 5 if row['LAW'] == 1 else 6 if row['LTD'] == 1 else 7 if row['PINC'] == 1 else 8 if row['TER'] == 1 else 9 if row['USE'] == 1 else 0),axis=1)
x_multi = df['text']
y_multi = df['labels']
df['labels'].value_counts(normalize=True)

labels
0    0.893324
6    0.029681
2    0.016849
8    0.015085
9    0.011853
3    0.010286
4    0.006661
5    0.006122
1    0.005192
7    0.004947
Name: proportion, dtype: float64

In [5]:
label2id = {
    'FAIR':0,
    'A':1,
    'CH':2,
    'CR':3,
    'J':4,
    'LAW':5,
    'LTD':6,
    'PINC':7,
    'TER':8,
    'USE':9
}
id2label = {v:k for k,v in label2id.items()}
id2label

{0: 'FAIR',
 1: 'A',
 2: 'CH',
 3: 'CR',
 4: 'J',
 5: 'LAW',
 6: 'LTD',
 7: 'PINC',
 8: 'TER',
 9: 'USE'}

# Binary Label Pre-Processing

In [6]:
x, y = df['text'], df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
rus = RandomUnderSampler(sampling_strategy=1)
x_train_res, y_train_res = rus.fit_resample(pd.DataFrame(x_train), pd.DataFrame(y_train))

In [7]:
y_train_res.value_counts()

label
0        1730
1        1730
Name: count, dtype: int64

In [8]:
train_df = pd.merge(x_train_res,y_train_res,left_index=True,right_index=True)
test_df = pd.merge(x_test,y_test,left_index=True,right_index=True)
train_df = train_df.sample(frac=1)
train_df.head()

,text,label
14972,when you set up a profile on the quora platfor...,0
187,"even after membership is terminated , the prop...",0
15246,• impersonates or misrepresents your affiliati...,0
16937,any election to arbitrate by one party shall b...,1
9886,without prejudice to the other provisions of t...,1


In [9]:
dataset_dict = DatasetDict(
    {
    "train":Dataset.from_pandas(train_df),
    "test":Dataset.from_pandas(test_df)
    }
)

Split dataset into train, test and validation

In [10]:
tokenizer = BertTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
def tokenize(batch):
    return tokenizer(batch['text'], max_length=512,padding='max_length')

In [11]:
tokenized_dict = dataset_dict.map(tokenize, batched=True, batch_size=len(dataset_dict))
tokenized_dict.set_format('torch', columns=['input_ids', 'attention_mask',"token_type_ids",'label'])

Map:   0%|          | 0/3460 [00:00<?, ? examples/s]

Map:   0%|          | 0/4084 [00:00<?, ? examples/s]

In [54]:
tokenized_dict['train'][0]

{'label': tensor(0),
 'input_ids': tensor([  101,  2043,  2017,  2275,  2039,  1037,  6337,  2006,  1996, 22035,
          2527,  4132,  1010,  2017,  2097,  2022,  2356,  2000,  3073,  3056,
          2592,  2055,  4426,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,   

Let's get the trainer ready

Now let's construct a custom classifier classifier to classify sentences as potentially unfair or not 

In [13]:
class Classifier_v1(nn.Module):
    def __init__(self, model_name):
        super(Classifier_v1,self).__init__()
        self.model = AutoModel.from_pretrained(model_name,config = AutoConfig.from_pretrained(model_name,
                                                                                              output_attention = True,
                                                                                              output_hidden_state = True))
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.hidden = nn.Linear(self.model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, input = None, attention_mask = None, token_type_ids = None,input_ids = None, labels = None):
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        pooler_output = outputs.pooler_output[0]
        output = self.hidden(pooler_output)
        logit = self.sigmoid(output)
        return logit

In [56]:
from torch.utils.data import DataLoader
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    tokenized_dict['train'], shuffle=True, batch_size=1, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_dict['test'], shuffle=True, batch_size=1,collate_fn=data_collator
)

In [57]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[  101,  2385,  1012,  1017,  1996,  4243, 13399,  2008,  6207,  6468,
          2053,  5368,  2005,  2151,  4447,  2008,  1996,  2224,  1997,  1996,
          3617,  4031,  1999, 19699, 23496,  2015,  1996,  7789,  3200,  2916,
          1997,  2353,  4243,  1012,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [47]:
tokenized_dict['train'][0]['label']

tensor(0)

## Training Loop

In [16]:
torch.cuda.empty_cache()

In [17]:
from torch.optim import Adam, SGD, lr_scheduler

model_v1 = Classifier_v1('microsoft/MiniLM-L12-H384-uncased').to(device)
epochs = 5
loss_fn = nn.BCELoss()
optimizer = Adam(model_v1.parameters(),lr =.0001)

# Full Loop

In [80]:
def train(model,train_dataloader,epochs,loss_fn,optimizer,lr_scheduler=None,testing=False):
    for epoch in range(epochs):
        model.train()
        for batch in tqdm.tqdm(train_dataloader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label = batch['labels'].to(torch.float32).to(device)
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            loss = loss_fn(output[0],label)
            loss.backward()
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step()
        preds = []
        labels = []
        for i,batch in tqdm.tqdm(enumerate(test_dataloader)):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label = batch['labels'].to(torch.float32).to(device)
            with torch.no_grad():
                output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            loss = loss_fn(output[0],label)
            pred = torch.round(output)
            preds.append(pred.detach().cpu().numpy()[0])
            labels.append(batch["labels"].detach().cpu().numpy())
        print(labels)
        print(preds)
        print(f"Epoch: {epoch + 1} | Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {loss}")

In [81]:
def test(model,test_dataloader,loss_fn):
    model.eval()
    preds = []
    labels = []
    for i,batch in tqdm.tqdm(enumerate(test_dataloader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label = batch['labels'].to(torch.float32).to(device)
        with torch.no_grad():
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        loss = loss_fn(output,label)
        pred = torch.round(output)
        preds.append(pred.detach().cpu().numpy()[0])
        labels.append(batch["labels"].detach().cpu().numpy())
    print(f"Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {loss}")
    return preds,labels

Looking much better than before, but there is still a lot of room for improvement

This is not working, even with oversampling we are still not getting good results. I think the problem is that the pooler output, which is the (mean?) of all the 12 hidden states of BERT that we are using as input to our additional classifier is not capturing the information we need to understand the fairness of a sentence. Instead of using the pooler output, let's use a concatenation of all the hidden states of the BERT model

In [91]:
class Classifier_V2(nn.Module):
    '''
    This model is similar to the first one but instead of using the pooler output, it uses the hidden states of the model
    The 'hidden_states_used' parameter is used to determine how many hidden states to use, smaller values of this will be less computationally expensive, but likely less accurate
    '''
    def __init__(self, model_name ,num_labels,hidden_states_used):
        super(Classifier_V2,self).__init__()
        self.hidden_states_used = hidden_states_used
        self.model = BertModel.from_pretrained(model_name,config = BertConfig.from_pretrained(model_name,output_hidden_states = True,num_labels=num_labels))
        self.hidden1 = nn.Linear(self.model.config.hidden_size*self.model.config.max_position_embeddings*self.hidden_states_used, 512)
        self.hidden2 = nn.Linear(512, num_labels)
        if num_labels == 1:
            self.activation = nn.Sigmoid()
        else:
            self.activation = nn.Softmax(dim=1)
    
    def forward(self, attention_mask = None, token_type_ids = None,input_ids = None):
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        hidden_states = torch.cat(outputs.hidden_states[-self.hidden_states_used:],dim=0).view(1,-1)
        output = self.hidden2(self.hidden1(hidden_states))
        logit = self.activation(output)
        return logit


In [96]:
torch.cuda.empty_cache()

In [95]:
from torch.optim import Adam, SGD, lr_scheduler

loss_fn = nn.BCELoss()
optimizer = Adam(model_v1.parameters(),lr =.01)
model_v2 = Classifier_V2('microsoft/MiniLM-L12-H384-uncased',1,6).to(device)

KeyboardInterrupt: 

In [94]:
train(model_v2,train_dataloader,5,loss_fn,optimizer)

  0%|          | 0/3460 [00:00<?, ?it/s]

  0%|          | 1/3460 [00:19<18:49:53, 19.60s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.75 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 8.32 GiB is allocated by PyTorch, and 80.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [63]:
for batch in train_dataloader:
    output = model_v2(input_ids=batch['input_ids'].to(device),attention_mask=batch['attention_mask'].to(device),token_type_ids=batch['token_type_ids'].to(device))
    print(batch['labels'])
    break
print(output[0].detach().cpu())

tensor([1])
tensor([0.4910])
